## Select glaciers within region shapes and basic attributes: Level 2 files

L0 and L1 files are too large: they are selected roughly by lon lat boxes and a buffer, so there is a lot of duplication. Here we select glaciers by region by overlaying the glaciers representative points with the region outlines. This reduces the size of the files and make them ready for regional processing.

In addition, we compute glacier area and store the CenLon, CenLat attributes for later use.

In [1]:
import utils 
import geopandas as gpd
import pandas as pd
import numpy as np
import shutil
import glob
import os
import subprocess
import tarfile
import shapely.geometry as shpg
import progressbar
import matplotlib.pyplot as plt
import warnings

In [6]:
def xy_coord(geom):
    x, y = rp.iloc[0].xy
    return x[0], y[0]

In [7]:
# go down from rgi7_scripts/workflow/preprocessing
data_dir = '../../../rgi7_data/'

In [8]:
reg_file = os.path.join(data_dir, 'l0_regions', '00_rgi70_regions', '00_rgi70_O1Regions.shp')
reg_f = gpd.read_file(reg_file)
# We drop the second alaska box which is useless
reg_f = reg_f.drop(1)

In [9]:
idir = utils.mkdir(data_dir + '/l1_with_interiors')
odir = utils.mkdir(data_dir + '/l2_sel_reg')
odir_tar = utils.mkdir(data_dir + '/l2_sel_reg_tars')

for reg in range(6, 8):

    if reg == 13:
        continue
    
    fp = f'{idir}/RGI{reg:02d}/RGI{reg:02d}.shp'
    print(fp)
    print('Reading...')
    shp = gpd.read_file(fp)
    
    print('Pointing...')
    
    rp = shp.representative_point()
    
    coordinates = np.array(list(rp.apply(xy_coord)))
    shp['CenLon'] = coordinates[:, 0]
    shp['CenLat'] = coordinates[:, 1]
    
    rp = rp.to_frame('geometry')
    rp['orig_index'] = shp.index
    
    intersect = gpd.overlay(rp, reg_f.loc[reg_f.RGI_CODE == f'{reg:02d}'], how='intersection')
    
    odf = shp.loc[intersect['orig_index']]
    
    print('Area-ing...')
    odf['area'] = odf.to_crs({'proj':'cea'}).area
    
    dd = utils.mkdir(f'{odir}/RGI{reg:02d}/', reset=True)
    
    print('Writing...')
    odf.to_file(dd + f'RGI{reg:02d}.shp')
    
    print('Taring...')
    print(subprocess.run(['tar', '-zcvf', f'{odir_tar}/RGI{reg:02d}.tar.gz', '-C', odir, f'RGI{reg:02d}']))

../../../rgi7_data//l1_with_interiors/RGI06/RGI06.shp
Reading...
Pointing...
Area-ing...
Writing...
Taring...
CompletedProcess(args=['tar', '-zcvf', '../../../rgi7_data//l2_sel_reg_tars/RGI06.tar.gz', '-C', '../../../rgi7_data//l2_sel_reg', 'RGI06'], returncode=0)
../../../rgi7_data//l1_with_interiors/RGI07/RGI07.shp
Reading...
Pointing...
Area-ing...
Writing...
Taring...
CompletedProcess(args=['tar', '-zcvf', '../../../rgi7_data//l2_sel_reg_tars/RGI07.tar.gz', '-C', '../../../rgi7_data//l2_sel_reg', 'RGI07'], returncode=0)
